In [1]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
import time

import logging

In [2]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

n_labels = len(labels)

labels_index = dict((j, i) for i, j in enumerate(labels))

labels_from_surround = dict( (l+'_surround', l) for l in labels[1:])

labels_surroundIncluded_list = labels[1:] + [l+'_surround' for l in labels[1:]]
labels_surroundIncluded = set(labels_surroundIncluded_list)

labels_surroundIncluded_index = dict((j, i) for i, j in enumerate(labels_surroundIncluded_list))

# colors = np.random.randint(0, 255, (len(labels_index), 3))

colors = np.loadtxt(os.environ['REPO_DIR'] + '/visualization/100colors.txt')
colors[labels_index['BackG']] = 1.

In [3]:
volume_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/'

In [4]:
downsample_factor = 16

section_thickness = 20 # in um
xy_pixel_distance_lossless = 0.46
xy_pixel_distance_tb = xy_pixel_distance_lossless * 32 # in um, thumbnail
# factor = section_thickness/xy_pixel_distance_lossless

xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
z_xy_ratio_downsampled = section_thickness / xy_pixel_distance_downsampled

In [ ]:
for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:

    section_bs_begin, section_bs_end = section_range_lookup[stack]
    print section_bs_begin, section_bs_end

    (volume_xmin, volume_xmax, volume_ymin, volume_ymax, volume_zmin, volume_zmax) = \
    np.loadtxt(os.path.join(volume_dir, 'volume_%(stack)s_scoreMap_limits.txt' % {'stack': stack}), dtype=np.int)

    volume2_allLabels = []

    for l in labels[1:]:

        print l

        volume2 = bp.unpack_ndarray_file(os.path.join(volume_dir, 'volume_%(stack)s_scoreMap_%(label)s.bp' % \
                                                      {'stack': stack, 'label': l}))

        volume2_cropped = volume2[volume_ymin:volume_ymax+1, volume_xmin:volume_xmax+1]
        # copy is important, because then you can delete the large array

        volume2_allLabels.append(volume2_cropped.copy())

        del volume2, volume2_cropped

    for l in range(1, n_labels):

        print labels[l]
        t = time.time()

        # compute gradient
        gy, gx, gz = np.gradient(volume2_allLabels[l-1], 10, 10, 10)
        save_hdf(gx, volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gx.hdf' % {'stack': stack, 'lab': labels[l]})
        save_hdf(gy, volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gy.hdf' % {'stack': stack, 'lab': labels[l]})
        save_hdf(gz, volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gz.hdf' % {'stack': stack, 'lab': labels[l]})

        sys.stderr.write('compute gradient: %f seconds\n' % (time.time() - t))

        t = time.time()

        # compute hessian
        gxy, gxx, gxz = np.gradient(gx, 10, 10, 10)    
        save_hdf(gxx, volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gxx.hdf' % {'stack': stack, 'lab': labels[l]})
        save_hdf(gxy, volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gxy.hdf' % {'stack': stack, 'lab': labels[l]})
        save_hdf(gxz, volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gxz.hdf' % {'stack': stack, 'lab': labels[l]})
        del gx, gxx, gxy, gxz

        gyy, gyx, gyz = np.gradient(gy, 10, 10, 10)
        save_hdf(gyx, volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gyx.hdf' % {'stack': stack, 'lab': labels[l]})
        save_hdf(gyy, volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gyy.hdf' % {'stack': stack, 'lab': labels[l]})
        save_hdf(gyz, volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gyz.hdf' % {'stack': stack, 'lab': labels[l]})
        del gy, gyx, gyy, gyz

        gzy, gzx, gzz = np.gradient(gz, 10, 10, 10)
        save_hdf(gzx, volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gzx.hdf' % {'stack': stack, 'lab': labels[l]})
        save_hdf(gzy, volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gzy.hdf' % {'stack': stack, 'lab': labels[l]})
        save_hdf(gzz, volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gzz.hdf' % {'stack': stack, 'lab': labels[l]})
        del gz, gzx, gzy, gzz

        sys.stderr.write('compute hessian: %f seconds\n' % (time.time() - t))